# DL Survival - Ventilation Outcomes
 Updated 21/11/21

In [2]:
import pandas as pd
import numpy as np
import math
import statistics
from datetime import datetime
import datetime as dt
from datetime import timedelta
import json
import miceforest as mf
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pd.set_option('display.max_columns', None)

## 1. Data cleaning

- Import MIMIC III data
- Review column unique values, assign correct data types
- Impute missing values


### 1.1: Importing data

In [3]:
df = pd.read_csv('mimic_combined.csv')

In [4]:
pd.set_option('display.max_columns', None)
print(df.shape)
df.head(5)

(12332, 110)


,Unnamed: 0,hb,subject_id,tricuspid,alt,plt,bg_temp,ventrate,insurance,creatinine,ckd,crp,insulin,specimen,hospital_expire_flag,ext_time,pud,los,admission_type,inr_1,sodium,hba1c,bilirubin_indirect,diab_cc,ethnicity,t2dm,dtoutput,fibrinogen,cabg,hadm_id,mi,admission_location,fio2,ffp,first_careunit,liver_mild,admittime,malig,pco2,height,baseexcess,rheum,po2,bleed_time,reint_time,plts,neutrophils,intime,bilirubin_direct,dischtime,bilirubin_total,infection,alp,ph,free_calcium,sbp,diab_un,hematocrit,magnesium,glucose,vent_array,smoking,cvd,pulmonary,tidalvol,chloride,rr,ggt,dod,t1dm,cryo,inr,potassium,pvd,aids,icustay_seq,copd,wcc,liver_severe,albumin,meanbp,gender,mit,dementia,arrhythmia,language,bicarb,hr,ptt,marital_status,bun,weight,reintubation,ast,last_careunit,deathtime,aortic,cardiac_index,ccf,lymphocytes,outtime,dbp,prbc,lactate,spo2,paraplegia,temp,pt,aado2,met_ca
0,0,"[{'charttime': datetime.datetime(2111, 2, 17, ...",29491,0,[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],[],Private,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,[],[],[],0,2111-02-16 13:00:00,0,1.0424,ELECTIVE,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],0,white,0,[],[],0,173798,0,PHYS REFERRAL/NORMAL DELI,[],[],CSRU,0,2111-02-16 07:15:00,0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",167.64,"[{'charttime': datetime.datetime(2111, 2, 16, ...",0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],NaN,[],[],2111-02-16 10:20:19,[],2111-02-21 15:49:00,[],"[{'suspected_infection_time': None, 'antibioti...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",0,[],[],"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'starttime': datetime.datetime(2111, 2, 16, ...",0,0,0,[],"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",[],NaN,0,[],"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",0,0,1,0,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,[],"[{'charttime': datetime.datetime(2111, 2, 16, ...",M,0,0,0,ENGL,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",[],DIVORCED,"[{'charttime': datetime.datetime(2111, 2, 17, ...",69.5,0,[],CSRU,NaN,1,[],0,[],2111-02-17 11:21:23,"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],"[{'charttime': datetime.datetime(2111, 2, 16, ...",0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],0
1,1,"[{'charttime': datetime.datetime(2192, 4, 20, ...",26775,0,[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],[],Medicare,"[{'charttime': datetime.datetime(2192, 4, 20, ...",0,[],[],[],0,2192-04-20 15:00:00,0,1.2855,ELECTIVE,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],[],0,white,0,[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",1,197759,0,PHYS REFERRAL/NORMAL DELI,[],[],CSRU,0,2192-04-20 07:15:00,0,"[{'charttime': datetime.datetime(2192, 4, 20, ...",157.48,"[{'charttime': datetime.datetime(2192, 4, 20, ...",0,"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],NaN,[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",2192-04-20 09:59:48,[],2192-04-24 13:23:00,[],"[{'suspected_infection_time': None, 'antibioti...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",0,"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'starttime': datetime.datetime(2192, 4, 20, ...",0,0,0,[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],NaN,0,[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",0,0,1,1,"[{'charttime': datet

In [5]:
# get bmi
def get_bmi(weight,height):
    bmi = (weight/(height**2))*10000
    return bmi
# generating bmi column
bmiList = []
for i in range(len(df)):
    y = get_bmi(df['weight'][i],df['height'][i])
    bmiList.append(y)
df['bmi'] = bmiList
df['bmi']

0        24.730307
1        33.064582
2        25.051320
3        29.158887
4        25.655469
           ...    
12327    21.947505
12328    26.592206
12329    26.606489
12330    26.218821
12331    24.981738
Name: bmi, Length: 12332, dtype: float64

#### 1.1.1: Column lists

In [45]:
#view and reorder columns
cols = list(df.columns)
new_cols = ['Unnamed: 0','hadm_id','subject_id','gender','ethnicity','marital_status','insurance','language','aortic','mit','tricuspid',
            'pulmonary','cabg','temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','bmi','cardiac_index','pt','ptt',
            'inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
            'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time','albumin',
            'creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate','po2','pco2','baseexcess','ph','aado2',
            'fio2','ffp','insulin','cryo','prbc','infection','ventrate','tidalvol','vent_array','reintubation','liver_severe','liver_mild',
            'rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd','paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi',
            'dementia','first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag','admittime',
            'dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime','plt','diab_un','diab_cc',
            'dtoutput','specimen','dod']

ptinfo=['Unnamed: 0','hadm_id','subject_id']

demographics=['gender','ethnicity','marital_status','insurance','language']

proceduretype=['aortic','mit','tricuspid','pulmonary','cabg']

vitals=['temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','bmi','cardiac_index']

labs=['pt','ptt','inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time',
'albumin','creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate']

bloodgases=['po2','pco2','baseexcess','ph','aado2','fio2']

products=['ffp','insulin','cryo','prbc','infection']

ventilation=['ventrate','tidalvol','vent_array','reintubation']

comorbidities=['liver_severe','liver_mild','rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd',
'paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi','dementia']

adm_cat=['first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag']

adm_num=['admittime','dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime']

others=['plt','diab_un','diab_cc','dtoutput','specimen','dod']

timeseries=[*vitals,*labs,*bloodgases,*products,*ventilation,'plt','dtoutput']
timeseries = [i for i in timeseries if i not in ('weight','height','bmi','reintubation', 'infection', 'vent_array')]
    
timeseries_valuenames = {'cardiac_index':'ci',
                         'plts':'bloodproduct',
                         'ffp':'bloodproduct',
                         'insulin':'amount',
                         'cryo':'bloodproduct',
                         'prbc':'bloodproduct',
                         'dtoutput':'output'}

In [7]:
df = df[new_cols]
df.head(10)

,Unnamed: 0,hadm_id,subject_id,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,temp,bg_temp,hr,spo2,rr,sbp,dbp,meanbp,weight,height,bmi,cardiac_index,pt,ptt,inr,inr_1,fibrinogen,hb,hematocrit,plts,wcc,lymphocytes,neutrophils,alp,ast,alt,ggt,bilirubin_indirect,bilirubin_direct,bilirubin_total,chloride,magnesium,potassium,crp,bleed_time,albumin,creatinine,free_calcium,sodium,bicarb,bun,hba1c,glucose,lactate,po2,pco2,baseexcess,ph,aado2,fio2,ffp,insulin,cryo,prbc,infection,ventrate,tidalvol,vent_array,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,admittime,dischtime,intime,outtime,ext_time,reint_time,los,icustay_seq,deathtime,plt,diab_un,diab_cc,dtoutput,specimen,dod
0,0,173798,29491,M,white,DIVORCED,Private,ENGL,1,0,0,0,0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",69.50,167.64,24.730307,[],[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],[],[],[],[],[],[],[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],[],[],[],[],"[{'suspected_infection_time': None, 'antibioti...",[],[],"[{'starttime': datetime.datetime(2111, 2, 16, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYS REFERRAL/NORMAL DELI,ELECTIVE,0,2111-02-16 07:15:00,2111-02-21 15:49:00,2111-02-16 10:20:19,2111-02-17 11:21:23,2111-02-16 13:00:00,NaN,1.0424,1,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,0,[],[],NaN
1,1,197759,26775,M,white,WIDOWED,Medicare,ENGL,1,0,0,0,1,"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",82.00,157.48,33.064582,"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],[],[],[],[],[],[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],[],[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[]

In [8]:
print(df[['tidalvol','vent_array']].iloc[2,0])
print(df[['tidalvol','vent_array']].iloc[2,1])

[{'charttime': datetime.datetime(2170, 1, 2, 8, 5), 'value': 350.0}
 {'charttime': datetime.datetime(2170, 1, 2, 9, 12), 'value': 490.0}
 {'charttime': datetime.datetime(2170, 1, 2, 10, 4), 'value': 480.0}
 {'charttime': datetime.datetime(2170, 1, 2, 12, 24), 'value': 340.0}
 {'charttime': datetime.datetime(2170, 1, 2, 14, 28), 'value': 300.0}]
[{'starttime': datetime.datetime(2170, 1, 1, 20, 0), 'endtime': datetime.datetime(2170, 1, 2, 13, 30), 'duration_hours': 17.5}]


### 1.2: Cleaning data types

#### 1.2.0: NaN assignment

In [9]:
df = df.replace('NaT',np.datetime64('NaT'))
df = df.replace(['[]','NaN',np.datetime64('NaT')],np.NaN)

#### 1.2.1: Datetime columns

In [10]:
# set column types as datetime
time_cols = ['admittime','dischtime','intime','outtime','reint_time','ext_time','deathtime']
for col in time_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')

#dod
df['dod'] = pd.to_datetime(df['dod'], format='%Y-%m-%d')

In [11]:
## CHECK FOR ROWS WHERE DEATHTIME < INTIME OR ADMITTIME

In [12]:
df[time_cols].dtypes

admittime     datetime64[ns]
dischtime     datetime64[ns]
intime        datetime64[ns]
outtime       datetime64[ns]
reint_time    datetime64[ns]
ext_time      datetime64[ns]
deathtime     datetime64[ns]
dtype: object

#### 1.2.2: Demographics

In [13]:
for x in demographics:
    print(x,': ',df[x].unique())

gender :  ['M' 'F']
ethnicity :  ['white' 'unknown' 'other' 'black' 'asian' 'hispanic' 'native' 'WHITE'
 'OTHER' 'UNKNOWN' 'UNABLE TO OBTAIN' 'ASIAN' 'HISPANIC/LATINO'
 'BLACK/AFRICAN AMERICAN' 'AMERICAN INDIAN/ALASKA NATIVE']
marital_status :  ['DIVORCED' 'WIDOWED' 'MARRIED' 'SINGLE' nan 'UNKNOWN (DEFAULT)'
 'SEPARATED']
insurance :  ['Private' 'Medicare' 'Medicaid' 'Government' 'Self Pay' 'Other']
language :  ['ENGL' nan 'CAPE' 'RUSS' 'MAND' 'PORT' 'CAMB' 'SPAN' 'PTUN' 'CANT' 'ITAL'
 'POLI' 'PERS' 'VIET' '*GUJ' 'URDU' '*BEN' 'KORE' 'ALBA' '*LEB' 'AMER'
 'GREE' 'HAIT' 'THAI' 'HIND' 'TURK' 'GERM' 'ARAB' 'ETHI' 'ENGLISH' '?']


In [14]:
#ethnicity
df.replace({'ethnicity':
                {'unknown': np.NaN,'UNKNOWN':np.NaN,'UNABLE TO OBTAIN':np.NaN,
                'OTHER':'other','WHITE':'white','BLACK/AFRICAN AMERICAN':'black','ASIAN':'asian',
                'HISPANIC/LATINO':'hispanic','AMERICAN INDIAN/ALASKA NATIVE':'native'
                }
            }, 
            inplace=True)
print(df['ethnicity'].unique())

['white' nan 'other' 'black' 'asian' 'hispanic' 'native']


In [15]:
#marital_status
df.replace({'marital_status':
                {'UNKNOWN (DEFAULT)': np.NaN
                }
            }, 
            inplace=True)
print(df['marital_status'].unique())

['DIVORCED' 'WIDOWED' 'MARRIED' 'SINGLE' nan 'SEPARATED']


In [16]:
#language
df.replace({'language':
                {'ENGLISH':'ENGL','?':np.NaN
                }
            }, 
            inplace=True)
print(df['marital_status'].unique())

['DIVORCED' 'WIDOWED' 'MARRIED' 'SINGLE' nan 'SEPARATED']


#### 1.2.3: ✔Procedure type

In [17]:
for x in proceduretype:
    print(x,': ',df[x].unique())

aortic :  [1 0]
mit :  [0 1]
tricuspid :  [0 1]
pulmonary :  [0 1]
cabg :  [0 1]


#### 1.2.4: **Vitals / Blood Gases / Products + infection / Ventilation


In [18]:
# wait for Jahan/others
# ventrate seems to be empty

#### 1.2.5: ✔Comorbidities

In [19]:
for x in comorbidities:
    print(x,': ',df[x].unique())

liver_severe :  [0 1]
liver_mild :  [0 1]
rheum :  [0 1]
cvd :  [0 1]
aids :  [0 1]
ckd :  [0 1]
copd :  [0 1]
arrhythmia :  [0 1]
pud :  [0 1]
smoking :  [0 1]
pvd :  [0 1]
paraplegia :  [0 1]
ccf :  [0 1]
met_ca :  [0 1]
t2dm :  [0 1]
t1dm :  [0 1]
malig :  [0 1]
mi :  [0 1]
dementia :  [0 1]


#### 1.2.6: Admissions (categorical)

In [20]:
for x in adm_cat:
    print(x,': ',df[x].unique())

first_careunit :  ['CSRU' 'CCU' 'SICU' 'MICU' 'TSICU'
 'Cardiac Vascular Intensive Care Unit (CVICU)' 'Coronary Care Unit (CCU)'
 'Medical Intensive Care Unit (MICU)'
 'Medical/Surgical Intensive Care Unit (MICU/SICU)' 'Trauma SICU (TSICU)'
 'Surgical Intensive Care Unit (SICU)'
 'Neuro Surgical Intensive Care Unit (Neuro SICU)' 'Neuro Intermediate']
last_careunit :  ['CSRU' 'TSICU' 'CCU' 'MICU' 'SICU'
 'Cardiac Vascular Intensive Care Unit (CVICU)' 'Coronary Care Unit (CCU)'
 'Medical Intensive Care Unit (MICU)'
 'Medical/Surgical Intensive Care Unit (MICU/SICU)' 'Trauma SICU (TSICU)'
 'Surgical Intensive Care Unit (SICU)'
 'Neuro Surgical Intensive Care Unit (Neuro SICU)']
admission_location :  ['PHYS REFERRAL/NORMAL DELI' 'TRANSFER FROM HOSP/EXTRAM'
 'CLINIC REFERRAL/PREMATURE' 'EMERGENCY ROOM ADMIT'
 'TRANSFER FROM OTHER HEALT' 'TRANSFER FROM SKILLED NUR'
 'PHYSICIAN REFERRAL' 'TRANSFER FROM HOSPITAL' 'EMERGENCY ROOM'
 'PROCEDURE SITE' 'WALK-IN/SELF REFERRAL' 'PACU'
 'INFORMATION N

In [21]:
#first_careunit
df.replace({'first_careunit':
                {'Cardiac Vascular Intensive Care Unit (CVICU)':'CVICU',
                'Coronary Care Unit (CCU)':'CCU',
                'Medical Intensive Care Unit (MICU)':'MICU',
                'Surgical Intensive Care Unit (SICU)':'SICU',
                'Neuro Intermediate':'Neuro Inter',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU/SICU',
                'Trauma SICU (TSICU)':'TSICU',
                'Neuro Surgical Intensive Care Unit (Neuro SICU)':'Neuro SICU'
                }
            }, 
            inplace=True)
print(df['first_careunit'].unique())

['CSRU' 'CCU' 'SICU' 'MICU' 'TSICU' 'CVICU' 'MICU/SICU' 'Neuro SICU'
 'Neuro Inter']


In [22]:
#last_careunit
df.replace({'last_careunit':
                {'Cardiac Vascular Intensive Care Unit (CVICU)':'CVICU',
                'Coronary Care Unit (CCU)':'CCU',
                'Medical Intensive Care Unit (MICU)':'MICU',
                'Surgical Intensive Care Unit (SICU)':'SICU',
                'Neuro Intermediate':'Neuro Inter',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU/SICU',
                'Trauma SICU (TSICU)':'TSICU',
                'Neuro Surgical Intensive Care Unit (Neuro SICU)':'Neuro SICU'
                }
            }, 
            inplace=True)
print(df['last_careunit'].unique())

['CSRU' 'TSICU' 'CCU' 'MICU' 'SICU' 'CVICU' 'MICU/SICU' 'Neuro SICU']


In [23]:
#admission_location
df.replace({'admission_location':
                {'TRANSFER FROM HOSP/EXTRAM':'TRANSFER FROM HOSPITAL',
                'PHYS REFERRAL/NORMAL DELI':'PHYSICIAN REFERRAL',
                'TRANSFER FROM SKILLED NUR':'TRANSFER FROM SKILLED NURSING FACILITY',
                'INFORMATION NOT AVAILABLE':np.NaN,
                'CLINIC REFERRAL':'CLINIC REFERRAL/PREMATURE',
                'EMERGENCY ROOM ADMIT':'EMERGENCY ROOM',
                }
            }, 
            inplace=True)
print(df['admission_location'].unique())

['PHYSICIAN REFERRAL' 'TRANSFER FROM HOSPITAL' 'CLINIC REFERRAL/PREMATURE'
 'EMERGENCY ROOM' 'TRANSFER FROM OTHER HEALT'
 'TRANSFER FROM SKILLED NURSING FACILITY' 'PROCEDURE SITE'
 'WALK-IN/SELF REFERRAL' 'PACU' nan 'AMBULATORY SURGERY TRANSFER'
 'INTERNAL TRANSFER TO OR FROM PSYCH']


#### 1.2.7: Others

In [24]:
# for x in others:
#     print(x,': ',df[x].unique())

### 1.3: Parsing time series data

In [25]:
df['vent_array'][14]

"[{'starttime': datetime.datetime(2138, 7, 25, 18, 0), 'endtime': datetime.datetime(2138, 7, 28, 6, 0), 'duration_hours': 60.0}\n {'starttime': datetime.datetime(2138, 7, 29, 12, 0), 'endtime': datetime.datetime(2138, 8, 5, 11, 30), 'duration_hours': 167.5}]"

In [26]:
def va_parser(row, output=6):
    """
    Takes row from `df` returns a list of starttime, endtime, vent duration 
    for first and (if applicable) second intubations

    Parameters
    ----------
    row : row in df
    output_ : select which output you want (use list index below) - e.g. args=[6] for all output when using df.apply()

    Returns
    -------
    single list variable containing  
        [0] int_time1: first intubation starttime
        [1] ext_time1: first intubation endtime
        [2] duration1: first intubation duration
        [3] int_time2: second intubation starttime
        [4] ext_time2: second intubation endtime 
        [5] duration2: second intubation duration
        [6] all

    """
    int_time1=np.NaN
    ext_time1=np.NaN
    duration1=np.NaN
    int_time2=np.NaN
    ext_time2=np.NaN
    duration2=np.NaN
    value = row['vent_array']
    list=[]
    '''a = value
    print(value)'''
    if value == np.NaN or pd.isna(value):
        return np.NaN
    a = value.replace("'",'"')
    a = a.replace('\n ...\n',',').replace('\n', ',').replace('...', '')
    a = a.replace('datetime.','"dt.')
    a = a.replace('),', ')",')
    a = json.loads(a)
    b = [(i['starttime'], i['endtime'], i['duration_hours']) for i in a]
    int_time1=dt.datetime.strptime(b[0][0],'dt.datetime(%Y, %m, %d, %H, %M)')
    ext_time1=dt.datetime.strptime(b[0][1],'dt.datetime(%Y, %m, %d, %H, %M)')
    duration1=b[0][2]
    
    if output==0:
        return int_time1
    if output==1:
        return ext_time1
    if output==2:
        return duration1

    if len(b)>=2:
        int_time2=dt.datetime.strptime(b[1][0],'dt.datetime(%Y, %m, %d, %H, %M)')
        ext_time2=dt.datetime.strptime(b[1][1],'dt.datetime(%Y, %m, %d, %H, %M)')
        duration2=b[1][2]
    if output==3:
        return int_time2
    if output==4:
        return ext_time2
    if output==5:
        return duration2
    if output==6:
        return int_time1, ext_time1, duration1, int_time2, ext_time2, duration2

In [27]:
df['int_time1']=df.apply(va_parser, args=[0], axis=1)
df['ext_time1']=df.apply(va_parser, args=[1], axis=1)
df['duration1']=df.apply(va_parser, args=[2], axis=1)
df['int_time2']=df.apply(va_parser, args=[3], axis=1)
df['ext_time2']=df.apply(va_parser, args=[4], axis=1)
df['duration2']=df.apply(va_parser, args=[5], axis=1)
df.head()

,Unnamed: 0,hadm_id,subject_id,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,temp,bg_temp,hr,spo2,rr,sbp,dbp,meanbp,weight,height,bmi,cardiac_index,pt,ptt,inr,inr_1,fibrinogen,hb,hematocrit,plts,wcc,lymphocytes,neutrophils,alp,ast,alt,ggt,bilirubin_indirect,bilirubin_direct,bilirubin_total,chloride,magnesium,potassium,crp,bleed_time,albumin,creatinine,free_calcium,sodium,bicarb,bun,hba1c,glucose,lactate,po2,pco2,baseexcess,ph,aado2,fio2,ffp,insulin,cryo,prbc,infection,ventrate,tidalvol,vent_array,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,admittime,dischtime,intime,outtime,ext_time,reint_time,los,icustay_seq,deathtime,plt,diab_un,diab_cc,dtoutput,specimen,dod,int_time1,ext_time1,duration1,int_time2,ext_time2,duration2
0,0,173798,29491,M,white,DIVORCED,Private,ENGL,1,0,0,0,0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",69.5,167.64,24.730307,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,NaN,NaN,NaN,NaN,NaN,"[{'suspected_infection_time': None, 'antibioti...",NaN,NaN,"[{'starttime': datetime.datetime(2111, 2, 16, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,2111-02-16 07:15:00,2111-02-21 15:49:00,2111-02-16 10:20:19,2111-02-17 11:21:23,2111-02-16 13:00:00,NaT,1.0424,1,NaT,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,0,NaN,NaN,NaT,2111-02-16 10:00:00,2111-02-16 13:00:00,3.0,NaT,NaT,NaN
1,1,197759,26775,M,white,WIDOWED,Medicare,ENGL,1,0,0,0,1,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",82.0,157.48,33.064582,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,NaN,NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'c

In [28]:
def inf_parser(value, timeLimits=None):
    if value == np.NaN or pd.isna(value):
        return 0
    a = value.replace("'", '"')
    a = a.replace('\n ...\n',',').replace('\n', ',').replace('...', '')
    a = a.replace('datetime.', '"dt.')
    a = a.replace('": None', '": "None"')
    for valuename in ['antibiotic','antibiotic_time']:
        a = a.replace(f'), "{valuename}"', f')", "{valuename}"')
    a = json.loads(a)
    b = [(eval(i['suspected_infection_time']), eval(i['antibiotic_time']), i['antibiotic'], i['specimen'], 
               i['positiveculture']) for i in a]
    sus = [i[0] for i in b]
    abx = [i[1] for i in b]
    pos = [i[4] for i in b]
    inf_ = 0
    for i in range(len(sus)):
        if sus[i]==None:
            sus[i]=dt.datetime(1000, 1, 1, 0, 0)
        if abx[i]==None:
            abx[i]=dt.datetime(1000, 1, 1, 0, 0)
        if ((sus[i]>= timeLimits[0] and sus[i]<=timeLimits[1]) or (abx[i]>= timeLimits[0] and abx[i]<=timeLimits[1])) and pos[i] == 1.0:
            inf_ = 1
            break
        else:
            continue
    return inf_

In [29]:
def ts_parser(value, timeDelta=None, timeLimits=None, int_time=None, valuename='value'):
    # timeDelta is timedelta in hours from earliest entry
    # timeLimits = (startTime, endTime)
    # if both timeDelta and timeLimits are provided, timeDelta overrules.
    # if both are None, then all timepoints are accepted
    if value == np.NaN or pd.isna(value):
        return np.NaN, np.NaN, np.NaN, np.NaN, np.NaN
    a = value.replace("'", '"')
    a = a.replace('\n ...\n',',').replace('\n', ',').replace('...', '')
    a = a.replace('datetime.', '"dt.')
    a = a.replace(f'), "{valuename}"', f')", "{valuename}"')
    a = a.replace('"unit": None', '"unit": "None"')
    a = a.replace('starttime', 'charttime')
    a = json.loads(a)
    b = [(eval(i['charttime']), i[valuename]) for i in a]

    if timeDelta:
        #startTime = min(b, key=lambda x:x[0])[0]
        startTime = int_time
        inc_b = [i[1] for i in b if i[0] <= startTime + dt.timedelta(hours=timeDelta)]
    else:
        if timeLimits:
            inc_b = [i[1] for i in b if i[0] >= timeLimits[0] and i[0] <= timeLimits[1]]
        else:
            inc_b = [i[1] for i in b]
    if len(inc_b) == 0:
        return np.NaN, np.NaN, np.NaN, np.NaN, np.NaN

    return sum(inc_b) / len(inc_b), max(inc_b), min(inc_b), inc_b[0], inc_b[-1]


In [30]:
# delete all rows where int_time1 OR ext_time1 are missing
df = df[~(pd.isnull(df['int_time1']) | pd.isnull(df['ext_time1']))]
df = df.reset_index()
df = df.drop(axis=1, columns=['index', 'Unnamed: 0'], inplace=False)
df

,hadm_id,subject_id,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,temp,bg_temp,hr,spo2,rr,sbp,dbp,meanbp,weight,height,bmi,cardiac_index,pt,ptt,inr,inr_1,fibrinogen,hb,hematocrit,plts,wcc,lymphocytes,neutrophils,alp,ast,alt,ggt,bilirubin_indirect,bilirubin_direct,bilirubin_total,chloride,magnesium,potassium,crp,bleed_time,albumin,creatinine,free_calcium,sodium,bicarb,bun,hba1c,glucose,lactate,po2,pco2,baseexcess,ph,aado2,fio2,ffp,insulin,cryo,prbc,infection,ventrate,tidalvol,vent_array,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,admittime,dischtime,intime,outtime,ext_time,reint_time,los,icustay_seq,deathtime,plt,diab_un,diab_cc,dtoutput,specimen,dod,int_time1,ext_time1,duration1,int_time2,ext_time2,duration2
0,173798,29491,M,white,DIVORCED,Private,ENGL,1,0,0,0,0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",69.50,167.64,24.730307,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,NaN,NaN,NaN,NaN,NaN,"[{'suspected_infection_time': None, 'antibioti...",NaN,NaN,"[{'starttime': datetime.datetime(2111, 2, 16, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,2111-02-16 07:15:00,2111-02-21 15:49:00,2111-02-16 10:20:19,2111-02-17 11:21:23,2111-02-16 13:00:00,NaT,1.042400,1,NaT,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,0,NaN,NaN,NaT,2111-02-16 10:00:00,2111-02-16 13:00:00,3.0,NaT,NaT,NaN
1,197759,26775,M,white,WIDOWED,Medicare,ENGL,1,0,0,0,1,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",82.00,157.48,33.064582,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,NaN,NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': 

### 1.4: Handling missing data

#### 1.4.0 Assessing for missing data

In [31]:
# check % missing values
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0: 'Missing Values', 1: '% Missing Values'})
    return mis_val_table_ren_columns

missing_data = missing_values_table(df)

In [32]:
#set limit and get list of variables missing above limit in `missing_cols`
# missing_limit = 50 #allen
missing_limit = 80 #marcel
missing_cols = missing_data.loc[missing_data['% Missing Values']>missing_limit].index.tolist()
print(missing_cols)
missing_data = missing_data.loc[missing_data['% Missing Values']>missing_limit]
missing_data = missing_data.sort_values(by=['% Missing Values'])
pd.set_option('display.max_rows', None)
missing_data

['plts', 'ggt', 'bilirubin_indirect', 'bilirubin_direct', 'crp', 'bleed_time', 'albumin', 'aado2', 'ffp', 'cryo', 'ventrate', 'reint_time', 'deathtime', 'dtoutput', 'specimen', 'dod', 'int_time2', 'ext_time2', 'duration2']


,Missing Values,% Missing Values
plts,7785,82.172261
albumin,7879,83.164450
ffp,7928,83.681655
aado2,8018,84.631623
dod,8059,85.064387
cryo,8567,90.426430
ext_time2,8845,93.360777
reint_time,8845,93.360777
int_time2,8845,93.360777
duration2,8845,93.360777


In [33]:
pd.reset_option('display.max_rows')

In [40]:
dfDroppedMissing = df[[i for i in df.columns if i not in missing_data.index]]
print(list(dfDroppedMissing.columns))

['hadm_id', 'subject_id', 'gender', 'ethnicity', 'marital_status', 'insurance', 'language', 'aortic', 'mit', 'tricuspid', 'pulmonary', 'cabg', 'temp', 'bg_temp', 'hr', 'spo2', 'rr', 'sbp', 'dbp', 'meanbp', 'weight', 'height', 'bmi', 'cardiac_index', 'pt', 'ptt', 'inr', 'inr_1', 'fibrinogen', 'hb', 'hematocrit', 'wcc', 'lymphocytes', 'neutrophils', 'alp', 'ast', 'alt', 'bilirubin_total', 'chloride', 'magnesium', 'potassium', 'creatinine', 'free_calcium', 'sodium', 'bicarb', 'bun', 'hba1c', 'glucose', 'lactate', 'po2', 'pco2', 'baseexcess', 'ph', 'fio2', 'insulin', 'prbc', 'infection', 'tidalvol', 'vent_array', 'reintubation', 'liver_severe', 'liver_mild', 'rheum', 'cvd', 'aids', 'ckd', 'copd', 'arrhythmia', 'pud', 'smoking', 'pvd', 'paraplegia', 'ccf', 'met_ca', 't2dm', 't1dm', 'malig', 'mi', 'dementia', 'first_careunit', 'last_careunit', 'admission_location', 'admission_type', 'hospital_expire_flag', 'admittime', 'dischtime', 'intime', 'outtime', 'ext_time', 'los', 'icustay_seq', 'plt'

In [43]:
#df = df.drop(axis=1, columns=list(missing_data.index), inplace=False) #allen
dfDroppedMissing = df.drop(axis=1, columns=[i for i in list(missing_data.index) if i not in ['aado2','fio2']]) #marcel
print(list(dfDroppedMissing.columns))

['hadm_id', 'subject_id', 'gender', 'ethnicity', 'marital_status', 'insurance', 'language', 'aortic', 'mit', 'tricuspid', 'pulmonary', 'cabg', 'temp', 'bg_temp', 'hr', 'spo2', 'rr', 'sbp', 'dbp', 'meanbp', 'weight', 'height', 'bmi', 'cardiac_index', 'pt', 'ptt', 'inr', 'inr_1', 'fibrinogen', 'hb', 'hematocrit', 'wcc', 'lymphocytes', 'neutrophils', 'alp', 'ast', 'alt', 'bilirubin_total', 'chloride', 'magnesium', 'potassium', 'creatinine', 'free_calcium', 'sodium', 'bicarb', 'bun', 'hba1c', 'glucose', 'lactate', 'po2', 'pco2', 'baseexcess', 'ph', 'aado2', 'fio2', 'insulin', 'prbc', 'infection', 'tidalvol', 'vent_array', 'reintubation', 'liver_severe', 'liver_mild', 'rheum', 'cvd', 'aids', 'ckd', 'copd', 'arrhythmia', 'pud', 'smoking', 'pvd', 'paraplegia', 'ccf', 'met_ca', 't2dm', 't1dm', 'malig', 'mi', 'dementia', 'first_careunit', 'last_careunit', 'admission_location', 'admission_type', 'hospital_expire_flag', 'admittime', 'dischtime', 'intime', 'outtime', 'ext_time', 'los', 'icustay_se

#### 1.4.1 Beginning imputation

In [50]:
dfForImpute = pd.DataFrame([0 for i in range(dfDroppedMissing.shape[0])])

# generating timeseries summary values
for column in timeseries:
    if column not in dfDroppedMissing.columns:
        continue
    x = timeseries_valuenames[column] if column in timeseries_valuenames else "value"
    meanList = []
    maxList = []
    minList = []
    firstList = []
    lastList = []
    for i in range(len(dfDroppedMissing[column])):
        inTime = dfDroppedMissing['int_time1'][i].to_pydatetime()
        extTime = dfDroppedMissing['ext_time1'][i].to_pydatetime() #marcel
        #y = ts_parser(dfDroppedMissing[column][i], timeLimits=(inTime, inTime+dt.timedelta(hours=12)), valuename=x)
        #y = ts_parser(dfDroppedMissing[column][i], timeLimits=(inTime, inTime+dt.timedelta(hours=4)), valuename=x) #survival_vent
        y = ts_parser(dfDroppedMissing[column][i], timeLimits=(inTime, extTime), valuename=x) #survival_revent
        meanList.append(y[0])
        maxList.append(y[1])
        minList.append(y[2])
        firstList.append(y[3])
        lastList.append(y[4])
    dfForImpute[column+'_mean'] = meanList
    dfForImpute[column+'_max'] = maxList
    dfForImpute[column+'_min'] = minList
    dfForImpute[column+'_first'] = firstList
    dfForImpute[column+'_last'] = lastList
dfForImpute = dfForImpute[[i for i in dfForImpute.columns if i != 0]]

C:\Users\marcel\AppData\Local\Temp/ipykernel_5436/1790794981.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfForImpute[column+'_last'] = lastList
C:\Users\marcel\AppData\Local\Temp/ipykernel_5436/1790794981.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfForImpute[column+'_mean'] = meanList
C:\Users\marcel\AppData\Local\Temp/ipykernel_5436/1790794981.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo

In [51]:
# generating infection during ventilation binary values

x = timeseries_valuenames[column] if column in timeseries_valuenames else "value"
infList = []
column='infection'
for i in range(len(df[column])):
    y = inf_parser(df[column][i], timeLimits=(df['int_time1'][i].to_pydatetime(), df['ext_time1'][i].to_pydatetime()))
    infList.append(y)
dfForImpute['infection_vent'] = infList

In [52]:
# add on non-time data for imputation
dfForImpute = dfForImpute.copy()
extraColumns = [i for i in dfDroppedMissing.columns if i not in list(dfForImpute.columns) + timeseries + ['infection', 'vent_array', 'int_time1', 'ext_time1'] + ptinfo + adm_num]
print(extraColumns)
for i in extraColumns:
    if i in ('weight', 'height', 'bmi','duration1'):
        dfForImpute[i] = df[i]
    else:
        dfForImpute[i] = df[i].astype('category')
#dfForImpute.drop(axis=1, columns=[0], inplace=True)
dfForImpute2 = dfForImpute.copy()

['gender', 'ethnicity', 'marital_status', 'insurance', 'language', 'aortic', 'mit', 'tricuspid', 'pulmonary', 'cabg', 'weight', 'height', 'bmi', 'reintubation', 'liver_severe', 'liver_mild', 'rheum', 'cvd', 'aids', 'ckd', 'copd', 'arrhythmia', 'pud', 'smoking', 'pvd', 'paraplegia', 'ccf', 'met_ca', 't2dm', 't1dm', 'malig', 'mi', 'dementia', 'first_careunit', 'last_careunit', 'admission_location', 'admission_type', 'hospital_expire_flag', 'diab_un', 'diab_cc', 'duration1']


In [53]:
# before imputation again
dfForImpute

,temp_mean,temp_max,temp_min,temp_first,temp_last,bg_temp_mean,bg_temp_max,bg_temp_min,bg_temp_first,bg_temp_last,hr_mean,hr_max,hr_min,hr_first,hr_last,spo2_mean,spo2_max,spo2_min,spo2_first,spo2_last,rr_mean,rr_max,rr_min,rr_first,rr_last,sbp_mean,sbp_max,sbp_min,sbp_first,sbp_last,dbp_mean,dbp_max,dbp_min,dbp_first,dbp_last,meanbp_mean,meanbp_max,meanbp_min,meanbp_first,meanbp_last,cardiac_index_mean,cardiac_index_max,cardiac_index_min,cardiac_index_first,cardiac_index_last,pt_mean,pt_max,pt_min,pt_first,pt_last,ptt_mean,ptt_max,ptt_min,ptt_first,ptt_last,inr_mean,inr_max,inr_min,inr_first,inr_last,inr_1_mean,inr_1_max,inr_1_min,inr_1_first,inr_1_last,fibrinogen_mean,fibrinogen_max,fibrinogen_min,fibrinogen_first,fibrinogen_last,hb_mean,hb_max,hb_min,hb_first,hb_last,hematocrit_mean,hematocrit_max,hematocrit_min,hematocrit_first,hematocrit_last,wcc_mean,wcc_max,wcc_min,wcc_first,wcc_last,lymphocytes_mean,lymphocytes_max,lymphocytes_min,lymphocytes_first,lymphocytes_last,neutrophils_mean,neutrophils_max,neutrophils_min,neutrophils_first,neutrophils_last,alp_mean,alp_max,alp_min,alp_first,alp_last,ast_mean,ast_max,ast_min,ast_first,ast_last,alt_mean,alt_max,alt_min,alt_first,alt_last,bilirubin_total_mean,bilirubin_total_max,bilirubin_total_min,bilirubin_total_first,bilirubin_total_last,chloride_mean,chloride_max,chloride_min,chloride_first,chloride_last,magnesium_mean,magnesium_max,magnesium_min,magnesium_first,magnesium_last,potassium_mean,potassium_max,potassium_min,potassium_first,potassium_last,creatinine_mean,creatinine_max,creatinine_min,creatinine_first,creatinine_last,free_calcium_mean,free_calcium_max,free_calcium_min,free_calcium_first,free_calcium_last,sodium_mean,sodium_max,sodium_min,sodium_first,sodium_last,bicarb_mean,bicarb_max,bicarb_min,bicarb_first,bicarb_last,bun_mean,bun_max,bun_min,bun_first,bun_last,hba1c_mean,hba1c_max,hba1c_min,hba1c_first,hba1c_last,glucose_mean,glucose_max,glucose_min,glucose_first,glucose_last,lactate_mean,lactate_max,lactate_min,lactate_first,lactate_last,po2_mean,po2_max,po2_min,po2_first,po2_last,pco2_mean,pco2_max,pco2_min,pco2_first,pco2_last,baseexcess_mean,baseexcess_max,baseexcess_min,baseexcess_first,baseexcess_last,ph_mean,ph_max,ph_min,ph_first,ph_last,aado2_mean,aado2_max,aado2_min,aado2_first,aado2_last,fio2_mean,fio2_max,fio2_min,fio2_first,fio2_last,insulin_mean,insulin_max,insulin_min,insulin_first,insulin_last,prbc_mean,prbc_max,prbc_min,prbc_first,prbc_last,tidalvol_mean,tidalvol_max,tidalvol_min,tidalvol_first,tidalvol_last,plt_mean,plt_max,plt_min,plt_first,plt_last,infection_vent,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,weight,height,bmi,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,diab_un,diab_cc,duration1
0,36.031745,36.444444,35.611110,36.166666,36.444444,NaN,NaN,NaN,NaN,NaN,89.000000,98.0,79.0,96.0,86.0,100.000000,100.0,100.0,100.0,100.0,14.230769,16.0,14.0,14.0,15.0,111.166667,123.0,90.0,117.0,105.0,54.750000,59.0,44.0,59.0,54.0,68.333333,74.0,55.0,74.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.900000,3.9,3.9,3.9,3.9,NaN,NaN,NaN,NaN,NaN,1.2800,1.28,1.28,1.28,1.28,137.0,137.0,137.0,137.0,137.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.000000,159.0,79.0,79.0,159.0,NaN,NaN,NaN,NaN,NaN,2.000000,2.0,2.0,2.0,2.0,45.000000,45.0,45.0,45.0,45.0,2.000000,2.0,2.0,2.0,2.0,7.390000,7.39,7.39,7.39,7.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,M,white,D

In [58]:
#dfForImpute2 = dfForImpute.drop(axis=1, columns=['duration1'])

kds = mf.ImputationKernel(
  dfForImpute2,
  datasets=1,
  save_all_iterations=True,
  random_state=1991
)

# Run the MICE algorithm for 3 iterations
kds.mice(3)

print(kds)

dfImputed = kds.complete_data(dataset=0, inplace=False)
print(dfImputed.isnull().sum(0))

# after imputation
dfImputed.head(10)

              Class: ImputationKernel
           Datasets: 1
         Iterations: 3
  Imputed Variables: 227
save_all_iterations: True
temp_mean               0
temp_max                0
temp_min                0
temp_first              0
temp_last               0
                       ..
admission_type          0
hospital_expire_flag    0
diab_un                 0
diab_cc                 0
duration1               0
Length: 262, dtype: int64


,temp_mean,temp_max,temp_min,temp_first,temp_last,bg_temp_mean,bg_temp_max,bg_temp_min,bg_temp_first,bg_temp_last,hr_mean,hr_max,hr_min,hr_first,hr_last,spo2_mean,spo2_max,spo2_min,spo2_first,spo2_last,rr_mean,rr_max,rr_min,rr_first,rr_last,sbp_mean,sbp_max,sbp_min,sbp_first,sbp_last,dbp_mean,dbp_max,dbp_min,dbp_first,dbp_last,meanbp_mean,meanbp_max,meanbp_min,meanbp_first,meanbp_last,cardiac_index_mean,cardiac_index_max,cardiac_index_min,cardiac_index_first,cardiac_index_last,pt_mean,pt_max,pt_min,pt_first,pt_last,ptt_mean,ptt_max,ptt_min,ptt_first,ptt_last,inr_mean,inr_max,inr_min,inr_first,inr_last,inr_1_mean,inr_1_max,inr_1_min,inr_1_first,inr_1_last,fibrinogen_mean,fibrinogen_max,fibrinogen_min,fibrinogen_first,fibrinogen_last,hb_mean,hb_max,hb_min,hb_first,hb_last,hematocrit_mean,hematocrit_max,hematocrit_min,hematocrit_first,hematocrit_last,wcc_mean,wcc_max,wcc_min,wcc_first,wcc_last,lymphocytes_mean,lymphocytes_max,lymphocytes_min,lymphocytes_first,lymphocytes_last,neutrophils_mean,neutrophils_max,neutrophils_min,neutrophils_first,neutrophils_last,alp_mean,alp_max,alp_min,alp_first,alp_last,ast_mean,ast_max,ast_min,ast_first,ast_last,alt_mean,alt_max,alt_min,alt_first,alt_last,bilirubin_total_mean,bilirubin_total_max,bilirubin_total_min,bilirubin_total_first,bilirubin_total_last,chloride_mean,chloride_max,chloride_min,chloride_first,chloride_last,magnesium_mean,magnesium_max,magnesium_min,magnesium_first,magnesium_last,potassium_mean,potassium_max,potassium_min,potassium_first,potassium_last,creatinine_mean,creatinine_max,creatinine_min,creatinine_first,creatinine_last,free_calcium_mean,free_calcium_max,free_calcium_min,free_calcium_first,free_calcium_last,sodium_mean,sodium_max,sodium_min,sodium_first,sodium_last,bicarb_mean,bicarb_max,bicarb_min,bicarb_first,bicarb_last,bun_mean,bun_max,bun_min,bun_first,bun_last,hba1c_mean,hba1c_max,hba1c_min,hba1c_first,hba1c_last,glucose_mean,glucose_max,glucose_min,glucose_first,glucose_last,lactate_mean,lactate_max,lactate_min,lactate_first,lactate_last,po2_mean,po2_max,po2_min,po2_first,po2_last,pco2_mean,pco2_max,pco2_min,pco2_first,pco2_last,baseexcess_mean,baseexcess_max,baseexcess_min,baseexcess_first,baseexcess_last,ph_mean,ph_max,ph_min,ph_first,ph_last,aado2_mean,aado2_max,aado2_min,aado2_first,aado2_last,fio2_mean,fio2_max,fio2_min,fio2_first,fio2_last,insulin_mean,insulin_max,insulin_min,insulin_first,insulin_last,prbc_mean,prbc_max,prbc_min,prbc_first,prbc_last,tidalvol_mean,tidalvol_max,tidalvol_min,tidalvol_first,tidalvol_last,plt_mean,plt_max,plt_min,plt_first,plt_last,infection_vent,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,weight,height,bmi,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,diab_un,diab_cc,duration1
0,36.031745,36.444444,35.611110,36.166666,36.444444,37.750000,38.2,37.6,37.7,37.6,89.000000,98.0,79.0,96.0,86.0,100.000000,100.0,100.0,100.0,100.0,14.230769,16.0,14.0,14.0,15.0,111.166667,123.0,90.0,117.0,105.0,54.750000,59.0,44.0,59.0,54.0,68.333333,74.0,55.0,74.0,66.0,2.592882,2.62443,2.54982,2.55605,2.75829,13.800000,17.4,12.7,14.7,12.8,41.966667,47.8,37.8,45.4,38.7,1.238889,1.6,1.2,1.3,1.2,1.240000,1.6,1.2,1.3,1.2,263.0,265.0,248.0,256.0,259.0,9.90000,10.9,9.0,10.0,9.0,30.800000,33.0,30.0,32.0,30.0,13.600000,14.4,13.2,9.2,14.1,15.400000,15.2,14.9,15.4,15.0,77.1,77.9,77.9,77.0,78.0,38.333333,47.0,31.0,31.0,45.0,65.333333,85.0,55.0,40.0,53.0,33.000000,42.0,14.0,34.0,27.0,1.000000,1.0,0.8,0.7,1.1,109.000000,111.0,107.0,111.0,108.0,3.1,3.5,2.8,3.5,3.1,3.900000,3.9,3.9,3.9,3.9,1.850000,2.3,1.1,1.1,2.0,1.280000,1.28,1.28,1.28,1.28,137.000000,137.0,137.0,137.0,137.0,22.500000,23.0,22.0,22.0,23.0,17.500000,19.0,15.0,17.0,19.0,5.5,5.5,5.5,5.5,5.5,95.000000,159.0,79.0,79.0,159.0,2.533333,2.8,2.2,2.8,2.4,2.000000,2.0,2.0,2.0,2.0,45.000000,45.0,45

In [59]:
y = np.asarray([i.to_pydatetime() for i in df["int_time1"]])
z = np.asarray([i.to_pydatetime() for i in df["outtime"]])
dfImputed['icu_stay_duration'] = [i.total_seconds() for i in z-y]
dfImputed[[i for i in adm_num if i != 'reint_time']] = df[[i for i in adm_num if i != 'reint_time']] #marcel
dfImputed[['hadm_id','subject_id']] = df[['hadm_id','subject_id']]
dfImputed[['int_time1', 'ext_time1', 'duration1','int_time2', 'ext_time2','duration2']] = df[['int_time1', 'ext_time1', 'duration1','int_time2', 'ext_time2','duration2']] #marcel not #allen
dfImputed

,temp_mean,temp_max,temp_min,temp_first,temp_last,bg_temp_mean,bg_temp_max,bg_temp_min,bg_temp_first,bg_temp_last,hr_mean,hr_max,hr_min,hr_first,hr_last,spo2_mean,spo2_max,spo2_min,spo2_first,spo2_last,rr_mean,rr_max,rr_min,rr_first,rr_last,sbp_mean,sbp_max,sbp_min,sbp_first,sbp_last,dbp_mean,dbp_max,dbp_min,dbp_first,dbp_last,meanbp_mean,meanbp_max,meanbp_min,meanbp_first,meanbp_last,cardiac_index_mean,cardiac_index_max,cardiac_index_min,cardiac_index_first,cardiac_index_last,pt_mean,pt_max,pt_min,pt_first,pt_last,ptt_mean,ptt_max,ptt_min,ptt_first,ptt_last,inr_mean,inr_max,inr_min,inr_first,inr_last,inr_1_mean,inr_1_max,inr_1_min,inr_1_first,inr_1_last,fibrinogen_mean,fibrinogen_max,fibrinogen_min,fibrinogen_first,fibrinogen_last,hb_mean,hb_max,hb_min,hb_first,hb_last,hematocrit_mean,hematocrit_max,hematocrit_min,hematocrit_first,hematocrit_last,wcc_mean,wcc_max,wcc_min,wcc_first,wcc_last,lymphocytes_mean,lymphocytes_max,lymphocytes_min,lymphocytes_first,lymphocytes_last,neutrophils_mean,neutrophils_max,neutrophils_min,neutrophils_first,neutrophils_last,alp_mean,alp_max,alp_min,alp_first,alp_last,ast_mean,ast_max,ast_min,ast_first,ast_last,alt_mean,alt_max,alt_min,alt_first,alt_last,bilirubin_total_mean,bilirubin_total_max,bilirubin_total_min,bilirubin_total_first,bilirubin_total_last,chloride_mean,chloride_max,chloride_min,chloride_first,chloride_last,magnesium_mean,magnesium_max,magnesium_min,magnesium_first,magnesium_last,potassium_mean,potassium_max,potassium_min,potassium_first,potassium_last,creatinine_mean,creatinine_max,creatinine_min,creatinine_first,creatinine_last,free_calcium_mean,free_calcium_max,free_calcium_min,free_calcium_first,free_calcium_last,sodium_mean,sodium_max,sodium_min,sodium_first,sodium_last,bicarb_mean,bicarb_max,bicarb_min,bicarb_first,bicarb_last,bun_mean,bun_max,bun_min,bun_first,bun_last,hba1c_mean,hba1c_max,hba1c_min,hba1c_first,hba1c_last,glucose_mean,glucose_max,glucose_min,glucose_first,glucose_last,lactate_mean,lactate_max,lactate_min,lactate_first,lactate_last,po2_mean,po2_max,po2_min,po2_first,po2_last,pco2_mean,pco2_max,pco2_min,pco2_first,pco2_last,baseexcess_mean,baseexcess_max,baseexcess_min,baseexcess_first,baseexcess_last,ph_mean,ph_max,ph_min,ph_first,ph_last,aado2_mean,aado2_max,aado2_min,aado2_first,aado2_last,fio2_mean,fio2_max,fio2_min,fio2_first,fio2_last,insulin_mean,insulin_max,insulin_min,insulin_first,insulin_last,prbc_mean,prbc_max,prbc_min,prbc_first,prbc_last,tidalvol_mean,tidalvol_max,tidalvol_min,tidalvol_first,tidalvol_last,plt_mean,plt_max,plt_min,plt_first,plt_last,infection_vent,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,weight,height,bmi,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,diab_un,diab_cc,duration1,icu_stay_duration,admittime,dischtime,intime,outtime,ext_time,los,icustay_seq,deathtime,hadm_id,subject_id,int_time1,ext_time1,int_time2,ext_time2,duration2
0,36.031745,36.444444,35.611110,36.166666,36.444444,37.750000,38.2,37.6,37.7,37.6,89.000000,98.0,79.0,96.0,86.0,100.000000,100.0,100.0,100.0,100.0,14.230769,16.0,14.0,14.0,15.0,111.166667,123.0,90.0,117.0,105.0,54.750000,59.0,44.0,59.0,54.0,68.333333,74.0,55.0,74.0,66.0,2.592882,2.62443,2.54982,2.55605,2.75829,13.800000,17.4,12.7,14.7,12.8,41.966667,47.8,37.8,45.4,38.7,1.238889,1.6,1.2,1.3,1.2,1.24,1.6,1.2,1.3,1.2,263.0,265.0,248.0,256.0,259.0,9.900000,10.9,9.0,10.0,9.0,30.800000,33.0,30.0,32.0,30.0,13.600000,14.4,13.2,9.2,14.1,15.400000,15.2,14.9,15.4,15.0,77.1,77.9,77.9,77.0,78.0,38.333333,47.0,31.0,31.0,45.0,65.333333,85.0,55.0,40.0,53.0,33.00,42.0,14.0,34.0,27.0,1.000000,1.0,0.8,0.7,1.1,109.000000,111.0,107.0,111.0,108.0,3.100000,3.5,2.8,3.5,3.1,3.900000,3.9,3.9,3.9,3.9,1.850000,2.3,1.1,1.1,2.0,1.2800,1.28,1.28,1.28,1.28,137.0,137.0,137.0,137.0,137.0,22.500000,23.0,22.0,22.0,23.0,1

In [60]:
#dfImputed.to_csv('imputed.csv')
#dfImputed.to_csv('imputed_vent.csv')
dfImputed.to_csv('imputed_revent.csv')